In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import sys
sys.path.append("../..")
import pyfolio
from pyfolio import timeseries
from copy import deepcopy
from config import (
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    VALID_1_START_DATE,
    VALID_1_END_DATE,
    VALID_2_START_DATE,
    VALID_2_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE
    
)

import warnings
warnings.filterwarnings('ignore')

/home/david/anaconda3/envs/torch_RL/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [3]:
price = pd.read_csv("../../dataset/daily-level/crypto_raw_t10_2023-04-01.csv")
price.rename(columns={"tic": "symbol", "timestamp": "Date"}, inplace=True)
price = price.set_index("Date")

In [4]:
def get_daily_return(df, value_col_name="portfolio_value"):
    df = deepcopy(df)
    df["daily_return"] = df[value_col_name].pct_change(1)
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df.set_index("timestamp", inplace=True, drop=True)
    df.index = df.index.tz_localize("UTC")
    return pd.Series(df["daily_return"], index=df.index)

In [5]:
def backtest_stats(account_value, value_col_name="portfolio_value"):
    dr_test = get_daily_return(account_value, value_col_name=value_col_name)
    perf_stats_all = timeseries.perf_stats(
        returns=dr_test,
        positions=None,
        transactions=None,
        turnover_denom="AGB",
    )
    return perf_stats_all

In [6]:
price

,open,high,low,volume,symbol,close
Date,,,,,,
2021-01-02,0.18134,0.18473,0.17000,4.622016e+08,ADAUSDT,0.17509
2021-01-03,0.17505,0.18457,0.16793,6.603033e+08,ADAUSDT,0.17742
2021-01-04,0.17740,0.20960,0.17255,1.201451e+09,ADAUSDT,0.20615
2021-01-05,0.20625,0.23992,0.19203,1.463416e+09,ADAUSDT,0.22528
2021-01-06,0.22518,0.26429,0.20696,1.478888e+09,ADAUSDT,0.25873
...,...,...,...,...,...,...
2023-03-29,0.47890,0.53400,0.46660,1.126346e+09,XRPUSDT,0.51440
2023-03-30,0.51450,0.58500,0.51450,1.291816e+09,XRPUSDT,0.54250
2023-03-31,0.54250,0.55800,0.52140,8.276913e+08,XRPUSDT,0.53190


In [7]:
price.iloc[3:5].close.to_list()

[0.22528, 0.25873]

In [8]:
daily = pd.pivot_table(price, values="close", index="Date", columns="symbol")

In [9]:
daily.columns

Index(['ADAUSDT', 'BNBUSDT', 'BTCUSDT', 'DOGEUSDT', 'ETCUSDT', 'ETHUSDT',
       'LTCUSDT', 'SOLUSDT', 'TRXUSDT', 'XRPUSDT'],
      dtype='object', name='symbol')

In [10]:
valid1 = daily[(daily.index >= VALID_1_START_DATE) & (daily.index < VALID_1_END_DATE)]
valid2 = daily[(daily.index >= VALID_2_START_DATE) & (daily.index < VALID_2_END_DATE)]
valid3 = daily[(daily.index >= TEST_START_DATE) & (daily.index < TEST_END_DATE)]

train1 = daily[daily.index < TRAIN_END_DATE]
train2 = daily[daily.index < VALID_1_END_DATE]
train3 = daily[daily.index < VALID_2_END_DATE]

In [11]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

mu = expected_returns.capm_return(train3)
S = risk_models.sample_cov(train3)
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.portfolio_performance(verbose=True)

Expected annual return: 69.1%
Annual volatility: 80.1%
Sharpe Ratio: 0.84


(0.690955017423698, 0.8014854447074872, 0.8371393664779178)

In [12]:
cleaned_weights

OrderedDict([('ADAUSDT', 0.1),
             ('BNBUSDT', 0.1),
             ('BTCUSDT', 0.1),
             ('DOGEUSDT', 0.1),
             ('ETCUSDT', 0.1),
             ('ETHUSDT', 0.1),
             ('LTCUSDT', 0.1),
             ('SOLUSDT', 0.1),
             ('TRXUSDT', 0.1),
             ('XRPUSDT', 0.1)])

In [13]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
latest_prices = get_latest_prices(train3)
weights = cleaned_weights
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=100000)

allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))
real_value = 0
for i in allocation.items():
    real_value += valid3.loc[valid3.iloc[-1].name, i[0]] * i[1]
print("Initial value: ${:.2f}".format(100000))
print("Investment value: ${:.2f}".format(real_value + leftover))


Discrete allocation: {'ADAUSDT': 40683, 'BNBUSDT': 40, 'DOGEUSDT': 146198, 'ETCUSDT': 643, 'ETHUSDT': 8, 'LTCUSDT': 147, 'SOLUSDT': 1010, 'TRXUSDT': 183857, 'XRPUSDT': 28977}
Funds remaining: $10583.48
Initial value: $100000.00
Investment value: $137956.05


# equal weight

In [15]:
10000 / valid3.iloc[0]

symbol
ADAUSDT      40683.482506
BNBUSDT         40.600893
BTCUSDT          0.604507
DOGEUSDT    142348.754448
ETCUSDT        637.348630
ETHUSDT          8.360295
LTCUSDT        142.571999
SOLUSDT       1003.009027
TRXUSDT     183486.238532
XRPUSDT      29507.229271
Name: 2023-01-01, dtype: float64

In [16]:
portfolio_value_list = []
allocation = 10000 / valid3.iloc[0]
for i in range(len(valid3)):
    tot = 0
    for j in allocation.items():
        tot += valid3.loc[valid3.iloc[i].name, j[0]] * j[1]
    portfolio_value_list.append(tot)
res = pd.DataFrame(portfolio_value_list, index=valid3.index, columns=["portfolio_value"])


In [17]:
res.reset_index(inplace=True)
res.rename(columns={"Date": "timestamp"}, inplace=True)

In [18]:
backtest_stats(res, value_col_name="portfolio_value")

Annual return          1.801536
Cumulative returns     0.444722
Annual volatility      0.516303
Sharpe ratio           2.275192
Calmar ratio           8.314937
Stability              0.176427
Max drawdown          -0.216663
Omega ratio            1.442527
Sortino ratio          3.795303
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.326499
Daily value at risk   -0.060387
dtype: float64